In [ ]:
%%writefile app.py

import streamlit as st
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time
import math
from collections import deque
import os
import csv

# ---------------- Configuration ----------------
CAM_INDEX = 0
MAX_PTS = 200
PINCH_PIXEL_THRESHOLD = 40
PINCH_NORM_THRESHOLD = 0.03
LOG_TO_CSV = False
CSV_FILENAME = "hand_distance_log.csv"
# ------------------------------------------------

# --- Helper Functions for Calculations ---

def calculate_distance(frame, hand_landmarks, img_w, img_h, draw=True):
    """
    Calculates and optionally draws the index-to-thumb distance.
    Returns (norm_dist, pixel_dist, pinch_status)
    """
    lm_thumb = hand_landmarks.landmark[4]
    lm_index = hand_landmarks.landmark[8]

    # Normalized distance (3D)
    dx_n = lm_thumb.x - lm_index.x
    dy_n = lm_thumb.y - lm_index.y
    dz_n = lm_thumb.z - lm_index.z
    norm_dist = math.sqrt(dx_n*dx_n + dy_n*dy_n + dz_n*dz_n)

    # Pixel distance (2D)
    tx_px = int(round(lm_thumb.x * img_w))
    ty_px = int(round(lm_thumb.y * img_h))
    ix_px = int(round(lm_index.x * img_w))
    iy_px = int(round(lm_index.y * img_h))
    pixel_dist = math.hypot(tx_px - ix_px, ty_px - iy_px)

    # Pinch detection
    pinch = pixel_dist <= PINCH_PIXEL_THRESHOLD or (norm_dist is not None and norm_dist <= PINCH_NORM_THRESHOLD)
    
    if draw:
        cv2.circle(frame, (tx_px, ty_px), 8, (0, 0, 255), -1)  # Red
        cv2.circle(frame, (ix_px, iy_px), 8, (255, 0, 0), -1)  # Blue
        cv2.line(frame, (tx_px, ty_px), (ix_px, iy_px), (0, 255, 0), 2) # Green
    
    return norm_dist, pixel_dist, pinch

# <<< NEW (Req 2)
def calculate_thumb_little_distance(frame, hand_landmarks, img_w, img_h):
    """Calculates and draws the thumb-to-little-finger distance."""
    lm_thumb = hand_landmarks.landmark[4]
    lm_little = hand_landmarks.landmark[20]

    # Normalized distance (3D)
    dx_n = lm_thumb.x - lm_little.x
    dy_n = lm_thumb.y - lm_little.y
    dz_n = lm_thumb.z - lm_little.z
    norm_dist = math.sqrt(dx_n*dx_n + dy_n*dy_n + dz_n*dz_n)

    # Pixel distance (2D)
    tx_px = int(round(lm_thumb.x * img_w))
    ty_px = int(round(lm_thumb.y * img_h))
    lx_px = int(round(lm_little.x * img_w))
    ly_px = int(round(lm_little.y * img_h))
    pixel_dist = math.hypot(tx_px - lx_px, ty_px - ly_px)
    
    # Draw on frame
    cv2.circle(frame, (tx_px, ty_px), 8, (0, 128, 255), -1)  # Orange
    cv2.circle(frame, (lx_px, ly_px), 8, (0, 128, 255), -1)  # Orange
    cv2.line(frame, (tx_px, ty_px), (lx_px, ly_px), (0, 128, 255), 2) # Orange
    
    return norm_dist, pixel_dist


def calculate_dimensions(frame, hand_landmarks, img_w, img_h):
    """Calculates and draws the hand's bounding box width and height."""
    x_coords = [int(round(lm.x * img_w)) for lm in hand_landmarks.landmark]
    y_coords = [int(round(lm.y * img_h)) for lm in hand_landmarks.landmark]
    
    min_x, max_x = min(x_coords), max(x_coords)
    min_y, max_y = min(y_coords), max(y_coords)
    
    width_px = max_x - min_x
    height_px = max_y - min_y
    
    aspect_ratio = width_px / height_px if height_px > 0 else 0
        
    # <<< MODIFIED (Req 3) - Draws lines at the boundaries
    # Width line (Magenta) at the top of the hand
    cv2.line(frame, (min_x, min_y), (max_x, min_y), (255, 0, 255), 2)
    # Height line (Cyan) at the left of the hand
    cv2.line(frame, (min_x, min_y), (min_x, max_y), (255, 255, 0), 2)
    
    return width_px, height_px, aspect_ratio

# --- Main App ---

def main():
    # --- Page Setup ---
    st.set_page_config(layout="wide")
    st.title("Hand Gesture Recognition and Distance Measurement")
    st.subheader("NAME: Hari Krishna Majji")
    st.subheader("Mentor: Dr. D. Bhanu Prakash")
    st.markdown("---")

    # --- Session State Initialization ---
    if 'camera_started' not in st.session_state:
        st.session_state.camera_started = False
    if 'show_distance' not in st.session_state:
        st.session_state.show_distance = False
    if 'show_dimensions' not in st.session_state:
        st.session_state.show_dimensions = False
    if 'show_thumb_little_dist' not in st.session_state: # <<< NEW (Req 2)
        st.session_state.show_thumb_little_dist = False

    # --- UI Layout ---
    # <<< MODIFIED (Req 1) - Start/Stop Button
    if st.session_state.camera_started:
        if st.button("Stop Camera"):
            st.session_state.camera_started = False
            st.rerun()
    else:
        if st.button("Start Camera"):
            st.session_state.camera_started = True
            st.rerun()
    
    st.markdown("---")

    # Create columns for Video and Plot
    vid_col, data_col = st.columns([2, 1])

    with vid_col:
        st.header("Live Webcam Feed")
        frame_placeholder = st.empty()
    
    with data_col:
        st.header("Live Data")
        # <<< NEW (Req 4 & 5) - "Always On" Placeholders
        handedness_placeholder = st.empty()
        pinch_placeholder = st.empty()
        st.markdown("---") # Visual separator
        
        # Placeholders for toggled data
        distance_output_placeholder = st.empty()
        thumb_little_dist_placeholder = st.empty() # <<< NEW (Req 2)
        dimensions_output_placeholder = st.empty()
        
        st.header("Live Plot (Thumb ↔ Index)")
        chart_placeholder = st.empty()

    # --- Toggles (only show if camera is on) ---
    if st.session_state.camera_started:
        st.header("Calculation Toggles")
        col1, col2, col3 = st.columns(3)
        with col1:
            st.session_state.show_distance = st.toggle("Thumb-Index Dist.", value=st.session_state.show_distance)
        with col2:
            st.session_state.show_thumb_little_dist = st.toggle("Thumb-Little Dist.", value=st.session_state.show_thumb_little_dist) # <<< NEW (Req 2)
        with col3:
            st.session_state.show_dimensions = st.toggle("Hand Dimensions", value=st.session_state.show_dimensions)

    # --- Camera and MediaPipe Setup ---
    if not st.session_state.camera_started:
        st.warning("Please click 'Start Camera' to begin.")
        st.stop()

    cap = cv2.VideoCapture(CAM_INDEX)
    if not cap.isOpened():
        st.error(f"Cannot open camera index {CAM_INDEX}.")
        st.stop()

    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(
        static_image_mode=False,
        model_complexity=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
        max_num_hands=2 # <<< MODIFIED (Req 4)
    )
    
    # Data structures for plotting
    timestamps = deque(maxlen=MAX_PTS)
    norm_dists = deque(maxlen=MAX_PTS)
    pixel_dists = deque(maxlen=MAX_PTS)
    start_time = time.time()
    
    # --- Main Loop ---
    try:
        while cap.isOpened():
            # <<< MODIFIED (Req 1) - Check stop button
            if not st.session_state.camera_started:
                break
                
            ret, frame = cap.read()
            if not ret:
                st.warning("Failed to read frame from camera. Stream ended.")
                break

            frame = cv2.flip(frame, 1)
            img_h, img_w = frame.shape[:2]
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)

            # --- Data Initialization for this frame ---
            handedness_text = ""
            pinch_text = ""
            plot_norm_dist, plot_pixel_dist = np.nan, np.nan # For Hand 0 plot
            
            # Clear toggled placeholders
            distance_output_placeholder.empty()
            thumb_little_dist_placeholder.empty()
            dimensions_output_placeholder.empty()

            if results.multi_hand_landmarks:
                
                # --- "Always On" Data Processing (All Hands) ---
                for hand_index, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    
                    # <<< MODIFIED (Req 4) - Get Handedness
                    try:
                        handedness = results.multi_handedness[hand_index].classification[0]
                        handedness_text += f"**Hand {hand_index} ({handedness.label}):** {handedness.score*100:.0f}%  \n"
                    except Exception:
                        handedness_text += f"**Hand {hand_index}:** N/A  \n"

                    # <<< MODIFIED (Req 5) - Always calc pinch (don't draw)
                    _, _, pinch = calculate_distance(frame, hand_landmarks, img_w, img_h, draw=False)
                    pinch_text += f"**Hand {hand_index} Pinch:** `{'YES' if pinch else 'no'}`  \n"
                    
                    # --- Base Landmark Drawing (All Hands) ---
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # --- Toggled Data Processing (FIRST Hand Only) ---
                # We use hand 0 for all specific calculations and plotting
                hand_0_landmarks = results.multi_hand_landmarks[0]
                
                # --- Button 1 Logic ---
                if st.session_state.show_distance:
                    plot_norm_dist, plot_pixel_dist, _ = calculate_distance(frame, hand_0_landmarks, img_w, img_h, draw=True)
                    distance_output_placeholder.markdown(
                        f"**Thumb-Index Distance (Hand 0):**\n"
                        f"- Norm: `{plot_norm_dist:.4f}`\n"
                        f"- Pixel: `{plot_pixel_dist:.1f} px`"
                    )
                    
                # --- Button 2 Logic (Thumb-Little) --- (Req 2)
                if st.session_state.show_thumb_little_dist:
                    tl_norm, tl_pix = calculate_thumb_little_distance(frame, hand_0_landmarks, img_w, img_h)
                    thumb_little_dist_placeholder.markdown(
                        f"**Thumb-Little Distance (Hand 0):**\n"
                        f"- Norm: `{tl_norm:.4f}`\n"
                        f"- Pixel: `{tl_pix:.1f} px`"
                    )

                # --- Button 3 Logic (Dimensions) --- (Req 3)
                if st.session_state.show_dimensions:
                    width_px, height_px, aspect_ratio = calculate_dimensions(frame, hand_0_landmarks, img_w, img_h)
                    dimensions_output_placeholder.markdown(
                        f"**Hand Dimensions (Hand 0):**\n"
                        f"- Width: `{width_px} px`\n"
                        f"- Height: `{height_px} px`\n"
                        f"- Aspect Ratio: `{aspect_ratio:.3f}`"
                    )

            # --- Update "Always On" UI Placeholders ---
            handedness_placeholder.markdown(handedness_text if handedness_text else "**Handedness:** No hands detected")
            pinch_placeholder.markdown(pinch_text if pinch_text else "**Pinch Status:** No hands detected")

            # --- Update Video Feed ---
            frame_for_display = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_placeholder.image(frame_for_display, channels="RGB")
            
            # --- Update Plotting Data (Only for Hand 0 Thumb-Index) ---
            elapsed = time.time() - start_time
            timestamps.append(elapsed)
            norm_dists.append(plot_norm_dist) # Will be np.nan if toggle is off
            pixel_dists.append(plot_pixel_dist) # Will be np.nan if toggle is off

            t_arr = np.array(timestamps)
            norm_arr = np.array(norm_dists)
            pix_arr = np.array(pixel_dists)

            # Scaling
            if np.nanmax(pix_arr) > 0 and np.nanmax(norm_arr) > 0:
                factor = (np.nanmax(norm_arr) + 1e-6) / (np.nanmax(pix_arr) + 1e-6)
            else:
                factor = 1.0
            scaled_pix = pix_arr * factor

            plot_df = pd.DataFrame({
                "Time (s)": t_arr,
                "Normalized Distance": norm_arr,
                "Scaled Pixel Distance": scaled_pix
            })
            
            chart_placeholder.line_chart(plot_df.set_index("Time (s)"))

    finally:
        cap.release()
        hands.close()
        # No need to reset state here, it's handled by the rerun
        print("Camera released.")

if __name__ == "__main__":
    main()

In [ ]:
!streamlit run app.py